## [Working title]

Authors: Mitchell, J. S., Anijärv, T-E., Can, Adem., Hermens, D.F., & Lagopoulos, J.

Code created by Jules Mitchell January 2024.

You are free to use this or any other code from this repository for your own projects and publications. Citation or reference to the repository is not required, but would be much appreciated (see more on README.md).

# Set-Up

In [ ]:
# Load packages
import networkx as nx
from idtxl import idtxl_io as io
import matplotlib.pyplot as plt
import numpy as np
import os 
import pickle
import pandas as pd

# Set directory
root_dir = os.getcwd()
derivatives_folder = "c:\\Users\\Bonnie\\OneDrive\\Documents\\GitHub\\IDTxl\\demos\\data\\derivatives"
results_folder = "c:\\Users\\Bonnie\\OneDrive\\Documents\\GitHub\\IDTxl\\demos\\data\\derivatives\\results"

# Specify lists for conditional statements
responders = ["sub-01", "sub-02"]

# Electrode names
electrode_names = pd.read_csv("C:/Users/Bonnie/OneDrive/Documents/GitHub/IDTxl/demos/data/sub-01/ses-01/EO_chan_activity.csv", header=None, usecols=[0])
electrode_names = list(electrode_names[0])

In [ ]:
# Define functions
# Mock Data
def generate_adjacency_matrix(Size, N_cells): # Generate a single NxN adjacency matrix with N random integer values
    matrix = np.zeros((Size, Size))
    indices = np.random.choice(Size*Size, N_cells, replace=False)
    matrix.flat[indices] = np.random.randint(1, 9, N_cells)
    return matrix

def generate_directed_graph(N_edges): # generate a single directed graph with N weighted edges
    G = nx.DiGraph() 
    edges_to_add = np.random.choice(range(6), size=(N_edges, 2), replace=False)
    for edge in edges_to_add:
        i, j = edge
        weight = np.random.uniform(0.1, 0.9)
        G.add_edge(i, j, weight=weight)
    return G

def generate_mock_graphs(option, num_graphs, nodes, prob): # Generate mock graphs with various models
    for i in range(num_graphs):
        if option == 1:
            # Erdos-Renyi graph
            graph = nx.erdos_renyi_graph(nodes, prob)
        elif option == 2:
            # Barabasi-Albert graph
            graph = nx.barabasi_albert_graph(nodes,2)
        elif option == 3:
            # Watts-Strogatz graph
            graph = nx.watts_strogatz_graph(nodes, 2, prob)
        elif option == 4:
            # Directed Erdos-Renyi graph
            graph = nx.fast_gnp_random_graph(nodes, prob, directed=True)
        else:
            # Default: Erdos-Renyi graph
            graph = nx.erdos_renyi_graph(nodes, 0.2)
    return graph

# Graph Manipulation 
def combine_graphs(all_graphs, normalise=False):
    average_graph = nx.DiGraph()
    
    N = len(all_graphs) if normalise else 1

    for graph in all_graphs: # may need to enumerate?
        for edge in graph.edges:
            source, target = edge
            if average_graph.has_edge(source, target):
                average_graph[source][target]['weight'] += graph[source][target]['weight']
            else:
                # If the edge doesn't exist in the average_graph, add it
                average_graph.add_edge(source, target, weight=graph[source][target]['weight'])
    
    # Normalize the edge weights after the loop
    for edge in average_graph.edges:
        source, target = edge
        average_graph[source][target]['weight'] /= N

    return average_graph

# Plotting
def graph_plot(graph, with_labels=False, with_weights=False):
    # Settings
    pos = nx.spring_layout(graph)
    with_labels = True if with_labels else False
    
    if not with_weights:
        edge_weights = 1.0
    else:
        edge_weights = [graph[edge[0]][edge[1]].get('weight', 1.0) for edge in graph.edges]

    # Drawing the graph with varying line thickness based on edge weights
    nx.draw_networkx(
        graph, 
        pos, 
        with_labels=with_labels, 
        node_size=700, 
        node_color='skyblue', 
        font_size=8, 
        font_color='black', 
        font_weight='bold', 
        edge_color='gray', 
        width=edge_weights,  # Set the edge thickness based on edge weights
        edge_cmap=plt.cm.Blues  # You can choose a colormap for the edges
    )

    plt.title('Graph Plot')
    plt.show()

# Load IDTxL results
def load_graphs(folder, responders): # change to load_bids
    # Initialize an empty list to store graphs
    all_results = []

    # Loop through subject folders
    for subject_folder in os.listdir(folder):
        subject_path = os.path.join(folder, subject_folder)
        
        # Check if the subject is a responder or non-responder
        if subject_folder in responders:
            response_category = "responder"
        else:
            response_category = "non_responder"

        # Loop through session folders
        for session_folder in os.listdir(subject_path):
            session_path = os.path.join(subject_path, session_folder)

            # Assuming graph files are stored as text files (modify as needed)
            results = [f for f in os.listdir(session_path) if f.endswith(".p")] # Adjust file type as required

            # Load each graph file into a directed graph
            for result in results:
                graph_file_path = os.path.join(session_path, result)

                # Load results class from file (modify based on your data format)
                idtxl_file = pickle.load(open(graph_file_path, 'rb'))

                # Convert to networkx graph    
                weights = 'te'
                adj_matrix = idtxl_file.get_adjacency_matrix(weights=weights, fdr=False, weight_type=float)
                network = io.export_networkx_graph(adjacency_matrix=adj_matrix, weights=weights)
                
                # Determine task type
                if 'EC' in graph_file_path:
                    task = 'EC'
                else:
                    task = 'EO'

                # Add the information to the list
                all_results.append({
                    "subject": subject_folder,
                    "response_category": response_category,
                    "timepoint": session_folder,
                    "task": task,
                    "results": idtxl_file,
                    "network": network,
                    "adj_matrix": adj_matrix,
                    'weights': weights
                })

    return all_results

# Global Network Measures
def calculate_global_network_measures(all_graphs):
    # Create an empty list to store data
    data = []

    # Loop through each loaded graph
    for graph_info in all_graphs:
        subject = graph_info["subject"]
        response_category = graph_info["response_category"]
        timepoint = graph_info["timepoint"]
        task = graph_info["task"]
        network = graph_info["network"]
        undirected_network = network.to_undirected() # Need to change each network to undirected to calculate small-world network and global efficiency

        # Global measures
        global_efficiency = nx.global_efficiency(undirected_network) # How efficiently information is exchanged (higher values indicate higher integration)
        #smallworld = nx.sigma(undirected_network)  # Balance of local clustering and shortest path length

        # Append data to the list
        data.append({
            "subject": subject,
            "response_category": response_category,
            "timepoint": timepoint,
            "task": task,
            "GEf": global_efficiency
            #"SM": smallworld
            })
   
    # Create a DataFrame from the list of data
    global_df = pd.DataFrame(data)

    return global_df

# Local Network Measures
def calculate_local_network_measures(all_graphs, electrode_names):
    # Create an empty list to store data
    clustering_temp = []
    betweenness_temp = []
    eigen_temp = []
    indegree_temp = []
    outdegree_temp = []

    # Loop through each loaded graph
    for graph_info in all_graphs:
        subject = graph_info["subject"]
        response_category = graph_info["response_category"]
        timepoint = graph_info["timepoint"]
        task = graph_info["task"]
        network = graph_info["network"]

        # Local measures
        clustering = nx.clustering(network)
        betweenness = nx.betweenness_centrality(network)
        # eigen = nx.eigenvector_centrality(network)
        indegree = nx.in_degree_centrality(network)
        outdegree = nx.out_degree_centrality(network)
        
        # Rename columns with electrode names (replace this with your electrode names)
        clustering_renamed = {electrode_names[i]: clustering.get(i, 0) for i in range(len(electrode_names))}
        betweenness_renamed = {electrode_names[i]: betweenness.get(i, 0) for i in range(len(electrode_names))}
        # eigen_renamed = {electrode_names[i]: eigen.get(i, 0) for i in range(len(electrode_names))}
        indegree_renamed = {electrode_names[i]: indegree.get(i, 0) for i in range(len(electrode_names))}
        outdegree_renamed = {electrode_names[i]: outdegree.get(i, 0) for i in range(len(electrode_names))}


        # Append data to the lists
        clustering_temp.append({
            "subject": subject,
            "response_category": response_category,
            "session": timepoint,
            "task": task,
            "measure": "ClCoef",
            ** clustering_renamed
            }) 
         
        betweenness_temp.append({
            "subject": subject,
            "response_category": response_category,
            "session": timepoint,
            "task": task,
            "measure": "Btwn",
            ** betweenness_renamed
            })  
        
        #eigen_temp.append({
            #"subject": subject,
            #"response_category": response_category,
            #"session": session,
            #"task": task,
            #** eigen_renamed
            #})  
        
        indegree_temp.append({
            "subject": subject,
            "response_category": response_category,
            "session": timepoint,
            "task": task,
            "measure": "InDgr",
            ** indegree_renamed
            })  
        
        outdegree_temp.append({
            "subject": subject,
            "response_category": response_category,
            "session": timepoint,
            "task": task,
            "measure": "OutDgr",
            ** outdegree_renamed
            })  
        
    # Create a DataFrame from the list of data
    clustering_df = pd.DataFrame(clustering_temp)
    betweenness_df = pd.DataFrame(betweenness_temp)
    #eigen_df = pd.DataFrame(eigen_temp)
    indegree_df = pd.DataFrame(indegree_temp)
    outdegree_df = pd.DataFrame(outdegree_temp)

    # Combine dataframes
    local_df = pd.concat([clustering_df, betweenness_df, indegree_df, outdegree_df], axis=0)

    # Set 'subject' column as the index
    local_df.set_index('subject', inplace=True)

    # Sort the DataFrame by the index
    local_df.sort_index(inplace=True)

    return local_df 

def te_extraction (all_graphs):

    # Initialize temporary variables
    subject_values = []
    task_values = []
    timepoint_values = []
    caudal_left_values = []
    rostral_left_values = []
    caudal_right_values = []
    rostral_right_values = []
    asymmetry_left_values = []
    asymmetry_right_values = []

    # Iterate through each participant-file in the list
    for participant in range(len(all_graphs)):

        # Initialise TE placeholders
        caudal_left_sum = 0
        caudal_right_sum = 0
        rostral_left_sum = 0
        rostral_right_sum = 0
        
        # Load participant details
        subject = all_graphs[participant]['subject']
        task = all_graphs[participant]['task']
        timepoint = all_graphs[participant]['timepoint']
        
        # Load the channel weights list per participant-file
        channel_weights_list = all_graphs[participant]['adj_matrix'].get_edge_list()
        
        # Iterate through the list and apply the conditional statement
        for i, j, weight in channel_weights_list:

            # Left hemisphere frontal to parietal: 0 (Fp1), 1 (AF3), 2 (F7), 3 (F3) --> 8 (CP1), 9 (CP5), 10 (P7), 11 (P3) 
            if (i == 0 or i == 1 or i == 2 or i == 3) and (j == 8 or j == 9 or j == 10 or j == 11):
                caudal_left_sum += weight

            # Right hemisphere frontal to parietal: 29 (Fp2), 28 (AF4), 27 (F8), 26 (F4) --> 21 (CP2), 20 (CP6), 19 (P8), 18 (P4) 
            if (i == 29 or i == 28 or i == 27 or i == 26) and (j == 21 or j == 20 or j == 19 or j == 18):
                caudal_right_sum += weight

            # Left hemisphere parietal to frontal: 8 (CP1), 9 (CP5), 10 (P7), 11 (P3) --> 0 (Fp1), 1 (AF3), 2 (F7), 3 (F3)
            if (i == 8 or i == 9 or i == 10 or i == 11) and (j == 0 or j == 1 or j == 2 or j == 3):
                rostral_left_sum += weight

            # Right hemisphere parietal to frontal: 21 (CP2), 20 (CP6), 19 (P8), 18 (P4) --> 29 (Fp2), 28 (AF4), 27 (F8), 26 (F4)
            if (i == 21 or i == 20 or i == 19 or i == 18) and (j == 29 or j == 28 or j == 27 or j == 26):
                rostral_right_sum += weight

        # Calculate the final values by scaling by the 1/number of nodes
        caudal_left_final_value = caudal_left_sum * (1/8)
        caudal_right_final_value = caudal_right_sum * (1/8)
        rostral_left_final_value = rostral_left_sum * (1/8)
        rostral_right_final_value = rostral_right_sum * (1/8)

        # Calculate asymmetry value
        if (caudal_left_final_value + rostral_left_final_value) == 0: 
            asymmetry_left = 0
        else:
            asymmetry_left = (caudal_left_final_value - rostral_left_final_value)/ (caudal_left_final_value + rostral_left_final_value)

        if (caudal_right_final_value + rostral_right_final_value) == 0: 
            asymmetry_right = 0
        else:
            asymmetry_right = (caudal_right_final_value - rostral_right_final_value)/ (caudal_right_final_value + rostral_right_final_value)

        # Append the values
        subject_values.append(subject)
        task_values.append(task)
        timepoint_values.append(timepoint)
        caudal_left_values.append(caudal_left_final_value)
        caudal_right_values.append(caudal_right_final_value)
        rostral_left_values.append(rostral_left_final_value)
        rostral_right_values.append(rostral_right_final_value)
        asymmetry_left_values.append(asymmetry_left)
        asymmetry_right_values.append(asymmetry_right)

        # Save the results to a DataFrame with an ID column
        te_extracted_df = pd.DataFrame({
            'Subject': subject_values,
            'Task': task_values,
            'Timepoint': timepoint_values, # add subject ID, task, timepoint
            'Caudal_Left': caudal_left_values,
            'Caudal_Right': caudal_right_values,
            'Rostral_Left': rostral_left_values,
            'Rostral_Right': rostral_right_values,
            'Asymmetry_Left': asymmetry_left_values,
            'Asymmetry_Right': asymmetry_right_values
            })

    return te_extracted_df  

# Mock Data Creation

###  Generate Mock data (Adjancency Matrices)

In [ ]:
# Specify size (N-by-N)
Size = 4

# Specify the number of cells you want to populate in each matrix (replace N_cells with your desired number)
N_cells = int(Size/2)

# Specify the number of matrices you want (replace N_matrices with your desired number)
N_matrices = 2

# Generate N_matrices adjacency matrices
adjacency_matrices = [generate_adjacency_matrix(Size, N_cells) for _ in range(N_matrices)]

# Print the generated adjacency matrices
for i, matrix in enumerate(adjacency_matrices):
    print(f"Adjacency Matrix {i + 1}:\n{matrix}\n")

# Print the resulting average adjacency matrix
print("Average Adjacency Matrix:")
print(np.mean(adjacency_matrices, axis=0))

### Generate Mock Data (Directed Graphs)

In [ ]:
# Specify the number of edges you want to populate in each graph (replace N_edges with your desired number)
N_edges = 2

# Specify the number of directed graphs you want (replace N_graphs with your desired number)
N_graphs = 2

# Generate N_graphs directed graphs
graphs = [generate_directed_graph(N_edges) for _ in range(N_graphs)]

# Print the generated graphs
for i, graph in enumerate(graphs):
    print(f"Directed Graph {i + 1}:\nNodes: {graph.nodes}\nEdges: {graph.edges(data=True)}\n")

### Generate Mock Data (Graph Options)

In [ ]:
# Optional mock graphs
graph = generate_mock_graphs(option=4, num_graphs=1, nodes=10, prob = 0.1)

# Post-Transfer Entropy Analyses

In [ ]:
# Load files (Note, there is currently an issue with the gml file load (something to do with strings))
all_graphs = load_graphs(derivatives_folder, responders)

In [ ]:
all_graphs

In [ ]:
# Verify results between single target and adj_matrix edge list
all_graphs[1]['results'].get_single_target(0, fdr=False)

In [ ]:
all_graphs[1]['adj_matrix'].get_edge_list() 

### TE Extraction

In [ ]:
# Extract TE values for significant source-target pairs and calculate information flow metrics
te_data = te_extraction(all_graphs)

In [ ]:
te_data

In [ ]:
# Export results as csv
os.chdir(results_folder)
te_data.to_csv('OKTOS_teMeasures.csv', index=False)

### Topographical Analysis

In [ ]:
# Global measures
global_data = calculate_global_network_measures(all_graphs)

In [ ]:
global_data

In [ ]:
# Local measures
local_data = calculate_local_network_measures(all_graphs, electrode_names)

In [ ]:
local_data

In [ ]:
# Export results as csv
os.chdir(results_folder)
global_data.to_csv('OKTOS_GlobalMeasures.csv', index=False)
local_data.to_csv('OKTOS_LocalMeasures.csv', index=False)

### Combine Graphs/Matrices

In [ ]:
# Combine directed graphs for responders and non-responders across timepoint and task
average_graph = combine_graphs(graphs, normalise = False)